In [1]:
import os
import cv2
import shutil
import requests
from configparser import ConfigParser
from sklearn.model_selection import train_test_split as sk

# Config

In [2]:
parser = ConfigParser()
parser.read('dev.ini')
#print(parser.sections())
#parser.options('settings')
end_folder_name = parser.get('settings', 'dest_name')
create_folder = parser.getboolean('settings', 'create_folder', fallback = True)
path= parser.get('settings', 'path', fallback = os.getcwd()) #+ '\\' + end_folder_name
th = parser.getint('settings', 'size_treshold', fallback = 9999)
timeout = parser.getint('settings', 'timeout', fallback = 20)
clean_src = parser.getboolean('settings', 'clean_src', fallback = False)
neuron_number = parser.getint('settings', 'neuron_number', fallback = 0)
train_split = parser.getfloat('settings', 'train_split' , fallback = 0.7)
val_split = parser.getfloat('settings', 'val_split' , fallback = 0.1)
test_split = parser.getfloat('settings', 'test_split' , fallback = 0.2)

In [3]:
path

'C:\\Users\\Julian\\Documents\\GitHub\\ppgn\\dataset'

# Get images and cut+resize

In [4]:
try:
    os.mkdir(os.path.join(path,end_folder_name))
    dest_folder_path = os.path.join(path,end_folder_name)
except:
    dest_folder_path = os.path.join(path,end_folder_name)
        
try:
    source = os.mkdir(os.path.join(path,'Source'))
except Exception as e:
    source = os.path.join(path,'Source')
    print(e)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'C:\\Users\\Julian\\Documents\\GitHub\\ppgn\\dataset\\Source'


In [5]:
dest_folder_path

'C:\\Users\\Julian\\Documents\\GitHub\\ppgn\\dataset\\cat'

In [6]:
#Cutting image function
def cutting(img):
    
    width, height = img.shape[:2]
    
    if width >= th or height>= th:
        pass
    else:
        if width > height:
            x = int((width - height)/2)
            img = img[x:x+height, 0:height]
        else:
            x = int((height - width)/2)
            img = img[0:width, x:x + width] 
    
    return img   

In [7]:
content = []
with open(os.path.join(path,'url.txt'),encoding="utf8") as ff:
    content = ff.readlines()

In [8]:
#Delete duplicates
content = list(set(content))

In [9]:
if clean_src:
    for item in os.listdir(source):
        os.remove(os.path.join(source, item))

In [10]:
i = 0

for num, url in enumerate(content): 
        
    try:
        img_data = requests.get(url, timeout = timeout).content
        
    except requests.exceptions.RequestException as e:
        print(e)
        continue
        
    try:
        
        i = str(num).zfill(5)
        name = os.path.join(source, str(i) + '.JPEG')
    
    
        with open(name, 'wb') as handler:
            handler.write(img_data)
  

        print(name)

        img = cv2.imread(name)
        
    
        img = cutting(img)

        
        res = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)

        cv2.imwrite(os.path.join(dest_folder_path, str(i) + '.JPEG'),res)
        
 
    except Exception as e:
        print(e)

C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00000.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00001.JPEG
'NoneType' object has no attribute 'shape'
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00002.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00003.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00004.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00005.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00006.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00007.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00008.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00009.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00010.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00011.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00012.JPEG
C:\Users\Julian\Documents\GitHub\ppgn\dataset\Source\00013.JPEG
'NoneType' object has no attribute 'shape'


In [11]:
ff.close()

# Create Name File

*First do manual cleaning*

In [12]:
#user_input = input('Did you clean pics manualy? [y/n]:')
user_input = 'y'

In [13]:
if user_input == 'y':
    content = os.listdir(dest_folder_path)
    #Create txt file to store image names
    f= open(dest_folder_path + '\\' + end_folder_name + "_img_names.txt","w+")
    for num, address in enumerate(content):
        i = str(num+1).zfill(5)
        f.write(end_folder_name + '_' +  i + '.JPEG ' + str(neuron_number) + '\n')

In [14]:
f.close()

# Train/Val/Test Split

In [15]:
all_pics = os.listdir(dest_folder_path)
all_pics = [file for file in all_pics if file.endswith('.JPEG')]

In [16]:
y = [neuron_number] * len(all_pics)
train, test, y_train, y_test = sk(all_pics, y, test_size=test_split, random_state=1)
train, val, y_val, y_val = sk(train, y_train, test_size=val_split, random_state=1)

In [17]:
#train folder
train.sort()

try:
    os.mkdir(os.path.join(dest_folder_path,'train'))
    pass
except Exception as e:
    print(e)
    
train_path = os.path.join(dest_folder_path,'train')


f= open(os.path.join(train_path,"_img_names.txt"),"w+")

    
for pic in train:
    try:
            
        shutil.move(os.path.join(dest_folder_path,pic), train_path)
        
        f.write(end_folder_name + '/' + str(pic) + ' ' + str(neuron_number) + '\n')
        
    except Exception as e:
        
        print(e)

f.close()

In [18]:
#validation folder
val.sort()

try:
    os.mkdir(os.path.join(dest_folder_path,'validation'))
    pass
except Exception as e:
    print(e)
    
val_path = os.path.join(dest_folder_path,'validation')


f= open(os.path.join(val_path,"_img_names.txt"),"w+")
    
    
for pic in val:
    try:
        
        shutil.move(os.path.join(dest_folder_path,pic), val_path)
        
        f.write(end_folder_name + '/' + str(pic) + ' ' + str(neuron_number) + '\n')
        
    except Exception as e:
        
        print(e)
        
f.close()

In [19]:
#train folder
test.sort()

try:
    os.mkdir(os.path.join(dest_folder_path,'test'))
    pass
except Exception as e:
    print(e)
    
    
test_path = os.path.join(dest_folder_path,'test')


f= open(os.path.join(test_path,"_img_names.txt"),"w+")
    
    
for pic in test:
    try:
        
        shutil.move(os.path.join(dest_folder_path,pic), test_path)
        
        f.write(end_folder_name + '/' + str(pic) + ' ' + str(neuron_number) + '\n')
         
    except Exception as e:
        
        print(e)
        
f.close()